# Web scraping
## An introduction using Beautiful Soup

This lesson will introduce you to the world of web scraping and show you how one method, Beautiful Soup, works to allow you to gather large amounts of data from all sorts of web pages. Basic HTML structure will also be introduced and you will learn how to navigate the "tree" structure. 
 

**Difficulty:** Intermediate

**Completion Time:** 45 Minutes

**Knowledge Required:** Python basics (variables, data types, functions, loops, list comprehensions), Pandas basics

**Knowledge Recommended:** HTML/webpage structure basic knowledge

**Learning Objectives:** After this lesson you will be able to:
1. Understand and implement web scraping using BeautifulSoup.
2. Identify and navigate HTML tree structure to isolate desired nodes.
3. Build the first steps of a text analysis pipeline.
4. Be aware of other resources and methods for web scraping.

**Research Pipeline:** 
1. Choose your resources - website(s) and volume of desired content
2. Use the skills in this lesson to obtain the data and turn it into a user-friendly dataframe 
3. Transform the data as best fits your project goals - once it is in a data frame many things are possible!

**Libraries Used:** 
- [Pandas](https://pandas.pydata.org) _For manipulating and organizing data_
- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) _For web scraping_   
- [Requests](https://pypi.org/project/requests/) _For accessing webpages_   
- [Urllib](https://pypi.org/project/urllib3/) _As an HTTP client_

## Introduction

**What is web scraping?**
- Web scraping refers to the process of extracting content (data) from a webpage using some form of software (bot, web crawler). 
- Utilizes the structure of web pages (which are built using HTML or XHTML, a text-based mark-up language) to extract the data you want.

**Why should I learn web scraping?**
- While there are many pre-compiled datasets out there, many projects require specific data that is not readily available. Being able to gather the precise data you want for your project is a huge advantage.
- Web scraping allows you to harvest very large amounts of data with ease.

**How do I learn web scraping?**
- There are many methods for web scraping, with some being more optimized for obtaining certain information. You can build your own, as we do in this lesson with Beautiful Soup, which allows you to obtain very specific data but must be modified for different sources. There is also pre-built software you can download, which can require little to no coding. This can be a browser extension or software run through the command line or through a user interface. 
- In this lesson we will focus on one particular method, but it is by no means the only option out there. 

## HTML, a brief overview

Before we get into web scraping we first need to understand what goes on behind the scenes of a webpage. Webpages are built from HTML or XHTML, which is a text based markup language. There is a hierarchy to HTML and once you understand how it works traversing it to find the information you need becomes simple.

### The hierarchy

The structure of HTML is often referred to as a tree because it is made up of one root element with any number of branching descendant nodes. For instance, a root `<html>` node may have two children, a `<head>` node and a `<body>` node; each of which contains its own children and their children. Therefore, the node directly below another in the tree structure is a child node while the entirety of its children and its childrens children are descendants of that node. Children can also have parents (the node above them) and siblings (those nodes on the same level as them). Thinking of it in these familial terms can make it more intuitive. Below is an image that illustrates this tree structure: 

<img src='https://www.dropbox.com/home/Ithaka%20web%20scraping%20images?preview=treeStructure.png' alt="The hierarchical tree structure of HTML" />

![ugh](/Independent Learning/treeStructure.png)

Simply put, elements are pairs of tags which tell the browser what information is stored between them and how to display it. For instance, `<p>` elements are used to contain paragraphs (textual data) and `<a>` elements contain links. There are many kinds of tags, but for our purposes most of the information we are looking for will be in `<p>`, `<a>` or `<span>` tags, which are typically children of `<div>` tags. You may notice that there are also words like `class` or `id` within these tags that have their own values. These may look confusing at first, but they are very helpful in helping to point to the exact node you want. 

### Finding the information you need

But how do you go about finding the node you need? In order to do this you need to do a bit of digging and leg work. To view this underlying structure you can navigate to the webpage you wish to scrape and hit `CTRL + right click` and then choose 'Inspect' or 'Inspect Element'. This will pull up a side or bottom panel that provides an interactive view of the underlying HTML. There is often an overwelming number of tags so it can be helpful to highlight a word in the main text you wish to obtain and then hitting `CTRL + right click` as this will take you directly to that location in the hierarchy. From here you can see the name of the element that contains the information you want along with its parent, which is the information you will need for scraping. 

In [1]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib

## Scraping: Getting Started

In this lesson we are going to scrape the Pitt News website and gather the articles that are currently on the home page. You can visit their website [here](https://pittnews.com/). We are going to begin by navigating to the home page and collecting all the links to the articles that exist there. 

In [2]:
# Storing the URL in a variable
baseUrl = 'https://pittnews.com/'

# Initialize an empty list for the links
hrefList = []

# Call the base URL, turn it into soup then find all parents of link elements, 
##  drill down to the children and append each link to the list
response = requests.get(baseUrl)
soup = BeautifulSoup(response.text, 'html.parser')
divs = soup.find_all('div', {'class':'sno-story-card-link'})
for div in divs:
    a = div.find('a')
    hrefList.append(a.get('href'))

In [3]:
# Check that it did what we think
print(len(hrefList))
print(hrefList[0])

55
https://pittnews.com/article/174298/news/not-on-campus-on-another-campus-about-130-pitt-first-years-to-live-in-carlow-university-dorms-this-academic-year/


### Scraping: Gathering Data

This chunk of code is a little more dense, so here is a brief explanation. Three empty dictionaries are initialized, one for the title of each article, one for the date it was published and one for the text of the article. Then we loop through each link from the list created above and open and collect the html from each page. This is turned to soup and from there each element we need is found and turned to text; which is then added as the value to its respective dictionary with the link to the article as the key. Using the links as keys allows us to utilize a very useful function later on when we are building the data frame. 

In [4]:
# get text and other details from articles
titleDict = {}
dateDict = {}
textDict = {}

for href in hrefList:
    response = requests.get(href)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        # Get the date/time for each article
        date = soup.find('span', {'class':'time-wrapper'}).text
        dateDict[href] = date
        
        # Get the text for each article
        content = soup.find('span', {'class':'storycontent'})
        paras = content.find_all('p')
        text = [t.text for t in paras]
        textDict[href] = text
        
        # Get the title of each article
        title = soup.find('h1', {'class':'storyheadline'}).text
        titleDict[href] = title
    except:
        print('There is an error with this link')  # So we know if there is an issue with a link
        pass

In [18]:
# Checking our collected data
# Turning it into a list first because dictionaries are not iterable

titleList = list(titleDict.items())
print(titleList[0])

dateList = list(dateDict.items()) 
print(dateList[0])

textList = list(textDict.items())
print([t[:3] for t in textList[7]])

('https://pittnews.com/article/174298/news/not-on-campus-on-another-campus-about-130-pitt-first-years-to-live-in-carlow-university-dorms-this-academic-year/', '‘Not on campus, on another campus’: About 130 Pitt first-years to live in Carlow University dorms this academic year')
('https://pittnews.com/article/174298/news/not-on-campus-on-another-campus-about-130-pitt-first-years-to-live-in-carlow-university-dorms-this-academic-year/', ' August 25, 2022')
['htt', ['Ah, Pittsburgh — it has us Panthers sweating buckets in sweltering heat one minute, then running to take cover from a torrential downpour the next. As fall semester kicks off, it’s just a matter of time before we’re walking to our classes in blistering cold.', 'Fall is the best season to spend as much time outside as possible. We’re fortunate enough to have some beautiful natural spots on our mostly urban campus, where we can enjoy some foliage in not-too-torturous temperatures.', 'With that in mind, this is a guide to some of

### Scraping: Cleaning the Data

Now that we have the data we want and have taken a quick look at it we are can there are a few things that need to be taken care of before the data is ready to be stored in a more final form. Firstly there are some strange special characters in the text (`\xa0`) which we want to remove. You may also have noticed that the text data is a list of strings, not one cohesive string. These strings should be joined into one before we move forward. 

This data is fairly clean, however this part of the data scraping pipeline can vary depending on the state of your data and the goal of your analysis. 

In [6]:
# Getting rid of the special character
for k,v in textDict.items():
    vrep = [x.replace('\xa0', '') for x in v]
    textDict[k] = vrep

# Joining the sentences 
textListCleaned = [' '.join(v) for k,v in textDict.items()]

### Scraping: Storing the Data

Now that the data is cleaned it can go in a data frame for futher manipulation or storage - whatever your project requires! 

In [14]:
# Title dictionary to dataframe
pittNewsDF = pd.DataFrame.from_dict(titleDict, orient='index')
# Reset the index so it is not the hrefs
pittNewsDF.reset_index(inplace=True)
# Name the columns
pittNewsDF.columns = ['Href', 'Title']
# Add our other dictionaries to the data frame using zip()
pittNewsDF['Date/Time'] = pittNewsDF['Href'].map(dateDict)
pittNewsDF['Text'] = textListCleaned
# Take a look
pittNewsDF.head()

,Href,Title,Date/Time,Text
0,https://pittnews.com/article/174298/news/not-o...,"‘Not on campus, on another campus’: About 130 ...","August 25, 2022",Noah Stephenson has a few complaints about his...
1,https://pittnews.com/article/174347/sports/to-...,"‘To finish first, first you must finish’: Pant...","August 26, 2022","On any given day during the school year, sever..."
2,https://pittnews.com/article/174308/news/bank-...,"Bank accounts, dining and more: SGB members ta...","August 26, 2022","As the fall semester begins, Pitt’s Student Go..."
3,https://pittnews.com/article/174339/sports/col...,Column | No need to worry about Pitt’s wide re...,"August 25, 2022",Replacing a starting wide receiver is a diffic...
4,https://pittnews.com/article/174349/opinions/o...,"Opinion | Your education is a privilege, not a...","August 26, 2022",Most university students know that feeling whe...


In [13]:
# Taking a quick look at the data frame
pittNewsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Href       40 non-null     object
 1   Title      40 non-null     object
 2   Date/Time  40 non-null     object
 3   Text       40 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


## Caveats and Future Directions

Congratulations! You now know the basics of web scraping and the mechanics of one scraping method. Armed with this knowledge, and perhaps a little documentation study, you are ready to tackle any website you wish and gather whatever data you need. However, before we get too carried away it is important to note that while Beautiful Soup is powerful there are some things it is not equipt to handle. 

**The Caveats**
- Beautiful Soup cannot handle JavaScript.
- It has to be tweaked for every new website, which does require some hands on work initially.
- Depends on having a parser and a library to send requests.
- Can be laggy and bad for huge quantities of data.
- Doesn't have good proxy support, so your IP address could be flagged.

**Some Other Options**

Since Beautiful Soup cannot handle everything (and maybe it is just not your favorite web scraping method) here are some other options that you might want to look into:
- [Selenium](https://www.selenium.dev): is a popular method that is able to handle JavaScript and AJAX requests and is very beginner friendly.
- [Scrapy](https://scrapy.org): an open source framework that is excellent at handling proxies and building pipelines for very large quantities of data. 

**Some Useful Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- 